In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer

# Carregar o conjunto de dados
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tcc/Netflix Dataset Latest 2021.csv")

# Selecionar as colunas relevantes
df = df[["Series or Movie", "Genre", "Tags", "Languages", "Country Availability"]]

# Remover linhas com valores ausentes
df.fillna({"Series or Movie": "unavailable", "Genre": "unavailable", "Tags": "unavailable", "Languages": "unavailable", "Country Availability": "unavailable"}, inplace=True)

# Converter as colunas categóricas em numéricas
label_encoder = LabelEncoder()
df["Series or Movie"] = label_encoder.fit_transform(df["Series or Movie"])

# Transformar a coluna "Genre" em variáveis binárias
mlb_genre = MultiLabelBinarizer()
genre_encoded = pd.DataFrame(mlb_genre.fit_transform(df["Genre"].str.split(", ")), columns=mlb_genre.classes_)

# Transformar a coluna "Languages" em variáveis binárias
mlb_languages = MultiLabelBinarizer()
languages_encoded = pd.DataFrame(mlb_languages.fit_transform(df["Languages"].str.split(", ")), columns=mlb_languages.classes_)

# Transformar a coluna "Tags" em variáveis binárias
mlb_tags = MultiLabelBinarizer()
tags_encoded = pd.DataFrame(mlb_tags.fit_transform(df["Tags"].str.split(", ")), columns=mlb_tags.classes_)

# Transformar a coluna "Country Availability" em variáveis binárias
mlb_country = MultiLabelBinarizer()
country_encoded = pd.DataFrame(mlb_country.fit_transform(df["Country Availability"].str.split(", ")), columns=mlb_country.classes_)

# Concatenar as variáveis binárias com o dataframe original
df = pd.concat([df, genre_encoded, languages_encoded, tags_encoded, country_encoded], axis=1)

# Remover as colunas originais
df.drop(["Genre", "Languages", "Tags", "Country Availability"], axis=1, inplace=True)

# Separar as variáveis de entrada e o alvo
X = df.drop("Series or Movie", axis=1)
y = df["Series or Movie"]

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Criando e treinando o modelo de redes neurais
model = Sequential()
model.add(Dense(16, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(loss='mean_squared_error', optimizer='adam')

model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

# Avaliando o modelo
loss = model.evaluate(X_test, y_test)
print('Erro médio quadrático:', loss)
print("\n=======================\n")

Epoch 1/50
236/236 [==============================] - 3s 7ms/step - loss: 0.1768
Epoch 2/50
236/236 [==============================] - 2s 8ms/step - loss: 0.1081
Epoch 3/50
236/236 [==============================] - 2s 7ms/step - loss: 0.0343
Epoch 4/50
236/236 [==============================] - 2s 10ms/step - loss: 0.0087
Epoch 5/50
236/236 [==============================] - 2s 10ms/step - loss: 0.0032
Epoch 6/50
236/236 [==============================] - 2s 7ms/step - loss: 0.0014
Epoch 7/50
236/236 [==============================] - 2s 7ms/step - loss: 9.0320e-04
Epoch 8/50
236/236 [==============================] - 2s 7ms/step - loss: 6.9467e-04
Epoch 9/50
236/236 [==============================] - 2s 7ms/step - loss: 5.7049e-04
Epoch 10/50
236/236 [==============================] - 2s 7ms/step - loss: 4.8715e-04
Epoch 11/50
236/236 [==============================] - 2s 7ms/step - loss: 4.7186e-04
Epoch 12/50
236/236 [==============================] - 2s 10ms/step - loss: 5.0765e-0

* X_train: É o conjunto de dados de treinamento, que contém as amostras de entrada para o modelo.

* y_train: São os rótulos correspondentes às amostras de treinamento, indicando as saídas esperadas para cada entrada.
epochs=50: Esse parâmetro define o número de épocas de treinamento, ou seja, quantas vezes o algoritmo de treinamento percorrerá todo o conjunto de dados.

* batch_size=32: Esse parâmetro define o tamanho do lote de amostras usado para atualizar os pesos do modelo durante o treinamento. Em outras palavras, o algoritmo de treinamento atualizará os pesos a cada lote de 32 amostras.

* verbose=1: Esse parâmetro controla o nível de verbosidade durante o treinamento. Um valor de 1 significa que serão exibidas informações de progresso durante o treinamento, como a perda (loss) e a acurácia (accuracy) em cada época.


In [ ]:
# Novo registro de dados a ser classificado
new_data = {
    "Genre": "Crime, Drama, Fantasy, Horror, Romance",
    "Tags": "Dramas,Comedies,Films Based on Books,British",
    "Languages": "English",
    "Country Availability": "United States"
}

# Criar um DataFrame com o novo registro de dados
new_data_df = pd.DataFrame(new_data, index=[0])

# Pré-processar o novo registro de dados
new_data_df.fillna({"Genre": "unavailable", "Tags": "unavailable", "Languages": "unavailable", "Country Availability": "unavailable"}, inplace=True)
genre_encoded = pd.DataFrame(mlb_genre.transform(new_data_df["Genre"].str.split(", ")), columns=mlb_genre.classes_)
languages_encoded = pd.DataFrame(mlb_languages.transform(new_data_df["Languages"].str.split(", ")), columns=mlb_languages.classes_)
tags_encoded = pd.DataFrame(mlb_tags.transform(new_data_df["Tags"].str.split(", ")), columns=mlb_tags.classes_)
country_encoded = pd.DataFrame(mlb_country.transform(new_data_df["Country Availability"].str.split(", ")), columns=mlb_country.classes_)
new_data_df = pd.concat([genre_encoded, languages_encoded, tags_encoded, country_encoded], axis=1)

# Fazer a previsão no novo registro de dados
prediction = model.predict(new_data_df)

# Interpretar a previsão
if prediction[0] == 0:
    print("O título é classificado como um filme.")
else:
    print("O título é classificado como uma série.")

1/1 [==============================] - 0s 111ms/step
O título é classificado como uma série.
